In [1]:
import torch 
from torch import nn
from d2l import torch as d2l

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [4]:
def num_gpus():  #@save
    '''Get the number of available GPUs'''
    return torch.cuda.device_count()

num_gpus()

1

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [7]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [12]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [17]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.7928, 0.4194, 0.2380],
        [0.0980, 0.5468, 0.3981]], device='cuda:0')

In [18]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [19]:
Y + Z

tensor([[1.7928, 1.4194, 1.2380],
        [1.0980, 1.5468, 1.3981]], device='cuda:0')

In [20]:
Z.cuda(0) is Z

True

In [21]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

c:\Users\alexa\Documents\Uni\Personal Projects\Dive-into-Deep-Learning\dive_into_deep_learning\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [22]:
net(X)

tensor([[1.5783],
        [1.5783]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
net[0].weight.data[0]

tensor([0.5015, 0.2281, 0.4960], device='cuda:0')

In [24]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model